# Analyse Exploratoire de Données (EDA)

In [ ]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import seaborn as sns
import ydata_profiling
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler, Normalizer

extract_path = './AI_Project_Data'

## Import des données

In [21]:
csv_employee_survey_data = os.path.join(extract_path, 'employee_survey_data.csv')
csv_manager_survey_data = os.path.join(extract_path, 'manager_survey_data.csv')
csv_general_data = os.path.join(extract_path, 'general_data.csv')
csv_out_time = os.path.join(extract_path, 'out_time.csv')
csv_in_time = os.path.join(extract_path, 'in_time.csv')

employee_survey_df = pd.read_csv(csv_employee_survey_data)
manager_survey_df = pd.read_csv(csv_manager_survey_data)
general_df = pd.read_csv(csv_general_data)
out_time_df = pd.read_csv(csv_out_time)
in_time_df = pd.read_csv(csv_in_time)

# init empty dataframe
work_info = pd.DataFrame()

## Nettoyage de données

### Valeurs dupliquées

In [ ]:
work_info.duplicated().sum()

### Valeurs constantes

In [23]:
def remove_constant_columns(df):
    return df.loc[:, df.nunique() > 1]

### Valeurs manquantes

In [24]:
def fill_categorical_na(df):
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    if len(categorical_cols) == 0:
        return df

    categorical_imputer = SimpleImputer(strategy='most_frequent')
    
    df.loc[:, categorical_cols] = categorical_imputer.fit_transform(df[categorical_cols])
    
    return df

In [25]:
def fill_numeric_na(df):
    for col in df.select_dtypes(include=['number', 'float64']).columns:
        median_value = df[col].median()
        df.loc[:, col] = df[col].fillna(round(median_value))
    return df

In [26]:
def fill_total_working_years(df):
    if 'TotalWorkingYears' in df.columns and 'YearsAtCompany' in df.columns:
        df.loc[:, 'TotalWorkingYears'] = df['TotalWorkingYears'].fillna(df['YearsAtCompany'])
    return df

### Type des valeurs

#### Simplification des types

In [27]:
def simplify_numeric_columns(df):
    df = df.apply(lambda col: col.astype(int) if col.dtype == 'float64' and col.dropna().mod(1).eq(0).all() else col)
    return df

#### Conversion des types `object` en valeurs numériques

In [28]:
def transform_attrition_to_bool(df):
    if 'Attrition' in df.columns:
        df.loc[:, 'Attrition'] = df['Attrition'].map({'Yes': True, 'No': False})
    return df

In [29]:
def encode_categorical_columns(df):
    categorical_cols = df.select_dtypes(include=['object']).columns
    if len(categorical_cols) == 0:
        return df
    
    if 'Attrition' in categorical_cols:
        categorical_cols = categorical_cols.drop('Attrition')

    encoder = OneHotEncoder(dtype=int, sparse_output=False)
    encoded_array = encoder.fit_transform(df[categorical_cols])

    encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(categorical_cols), index=df.index)
    
    df = df.drop(columns=categorical_cols).join(encoded_df)
    return df

#### Gestion des fichiers contenant les horaires d'entrée et sortie des salariés

In [30]:
def reverse_and_merge(df):
    in_time_melted = in_time_df.melt(id_vars=['Unnamed: 0'], var_name='date', value_name='arrival_time')
    out_time_melted = out_time_df.melt(id_vars=['Unnamed: 0'], var_name='date', value_name='departure_time')

    # Renommer la colonne EmployeeID
    in_time_melted.rename(columns={'Unnamed: 0': 'EmployeeID'}, inplace=True)
    out_time_melted.rename(columns={'Unnamed: 0': 'EmployeeID'}, inplace=True)

    # Fusionner les deux DataFrames sur 'id' et 'date'
    merged_clock_in = pd.merge(in_time_melted, out_time_melted, on=['EmployeeID', 'date'], how='outer')
    return merged_clock_in

In [31]:
def generate_work_column(df):
    df['arrival_time'] = pd.to_datetime(df['arrival_time'])
    df['departure_time'] = pd.to_datetime(df['departure_time'])

    # Calculer le temps travaillé (différence entre départ et arrivée)
    df['worked_time'] = df['departure_time'] - df['arrival_time']

    # Convertir en heures pour avoir un format lisible
    df['worked_hours'] = df['worked_time'].dt.total_seconds() / 3600

    # Trier par id et date
    df.sort_values(by=['EmployeeID', 'date'], inplace=True)

    # Moyenne de la durée de travail par jour pour chaque employé
    mean_worked_hours = df.groupby('EmployeeID')['worked_hours'].mean()

    # Nombre total d'heures travaillées par employé
    total_worked_hours = df.groupby('EmployeeID')['worked_hours'].sum()

    # Nombre de jours ou le worked_hours est non nul
    worked_days = df[df['worked_hours'] > 0].groupby('EmployeeID')['worked_hours'].count()

    # Faire un data frame avec ces 3 informations pour chaque employé avec l'EmployeeID comme index
    work_info = pd.concat([mean_worked_hours, total_worked_hours, worked_days], axis=1)
    work_info.columns = ['mean_worked_hours', 'total_worked_hours', 'worked_days']
    return work_info

### Pipeline

In [32]:
num_attribs = work_info.select_dtypes(include=[np.number])

default_preprocessor = Pipeline([
    ('remove_constants', FunctionTransformer(remove_constant_columns, validate=False)),
    ('fill_categorical_na', FunctionTransformer(fill_categorical_na, validate=False)),
    ('fill_numeric_na', FunctionTransformer(fill_numeric_na, validate=False)),
    ('encode_categorical_columns', FunctionTransformer(encode_categorical_columns, validate=False)),
    ('simplify_numeric_columns', FunctionTransformer(simplify_numeric_columns, validate=False)),
])

employee_survey_preprocessor = Pipeline([
    ('default_preprocessor', default_preprocessor),
])

manager_survey_preprocessor = Pipeline([
    ('default_preprocessor', default_preprocessor),
])

general_preprocessor = Pipeline([
    ('fill_total_working_years', FunctionTransformer(fill_total_working_years, validate=False)),
    ('transform_attrition_to_bool', FunctionTransformer(transform_attrition_to_bool, validate=False)),
    ('default_preprocessor', default_preprocessor),
])

work_info_preprocessor = Pipeline([
    ('reverse_and_merge', FunctionTransformer(reverse_and_merge, validate=False)),
    ('generate_work_column', FunctionTransformer(generate_work_column, validate=False)),
])

# Nettoyage des données
employee_survey_data = employee_survey_preprocessor.fit_transform(employee_survey_df)
manager_survey_data = manager_survey_preprocessor.fit_transform(manager_survey_df)
general_data = general_preprocessor.fit_transform(general_df)
work_info = work_info_preprocessor.fit_transform(work_info)

### Fusion des fichiers

In [33]:
full_data = general_data.merge(employee_survey_data, on='EmployeeID').merge(manager_survey_data, on='EmployeeID').merge(work_info, on='EmployeeID')

## Analyse

### YDataProfiling : résumé des différents graphiques

In [34]:
# full_data_report = ydata_profiling.ProfileReport(full_data, title='Full Data')
# full_data_report.to_notebook_iframe()
# TODO : enlever le commentaire

## Exportation du dataset modifié

In [35]:
full_data.to_csv(os.path.join(extract_path, 'cleaned_data.csv'), index=False)

## Sélection des caractéristiques ayant le plus d'impact sur le taux d'attrition

In [36]:
from sklearn.feature_selection import SelectKBest, VarianceThreshold, chi2, SelectFromModel, RFE, RFECV
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
from collections import defaultdict

def compare_feature_selection_methods(X, y, k_features):
    # Dictionnaire pour stocker les features sélectionnées par chaque méthode
    selected_features = defaultdict(list)
    
    # VarianceThreshold
    selector = VarianceThreshold(threshold=0.5)
    selector.fit_transform(X)
    selected_features['VarianceThreshold'] = list(X.columns[selector.get_support()])
    
    # SelectKBest
    selector = SelectKBest(chi2, k=k_features)
    selector.fit_transform(X, y)
    selected_features['SelectKBest'] = list(X.columns[selector.get_support()])
    
    # SelectFromModel
    selector = SelectFromModel(RandomForestClassifier(n_estimators=100))
    selector.fit_transform(X, y)
    selected_features['SelectFromModel'] = list(X.columns[selector.get_support()])
    
    # RFE
    selector = RFE(RandomForestClassifier(n_estimators=100), 
                  n_features_to_select=k_features, 
                  step=1)
    selector.fit_transform(X, y)
    selected_features['RFE'] = list(X.columns[selector.get_support()])
    
    # RFECV
    selector = RFECV(RandomForestClassifier(n_estimators=100), 
                    min_features_to_select=k_features,
                    step=1, 
                    cv=5)
    selector.fit_transform(X, y)
    selected_features['RFECV'] = list(X.columns[selector.support_])
    
    # Créer un DataFrame avec le nombre maximal de caractéristiques
    max_features = max(len(features) for features in selected_features.values())
    
    # Remplir avec None pour avoir des colonnes de même longueur
    for method in selected_features:
        selected_features[method].extend([None] * (max_features - len(selected_features[method])))
    
    # Créer le DataFrame final
    features_table = pd.DataFrame(selected_features)
    
    # Ajouter des statistiques sur les features sélectionnées
    print("\nNombre de caractéristiques sélectionnées par méthode:")
    for method in selected_features:
        count = sum(1 for x in selected_features[method] if x is not None)
        print(f"{method}: {count} features")
    
    return features_table

In [ ]:
# Utilisation
X = full_data.drop(columns=['EmployeeID', 'Attrition'])
y = full_data['Attrition'].astype(int)

features_table = compare_feature_selection_methods(X, y, k_features=15)
print("\nTableau des caractéristiques sélectionnées:")
features_table

# Sélectionner le modèle le plus performant pour prédire l'attrition

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd 
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression


# Load the data
filename = './AI_Project_Data/cleaned_data.csv'
employee_data = pd.read_csv(filename)

# Displaying data.head() to see the first 5 rows of the data
employee_data.head()

## Prédiction des valeurs d'attrition

### Séparation du dataset en plusieurs échantillons pour la validation croisée

In [39]:
from sklearn.model_selection import StratifiedShuffleSplit
employee_df = employee_data.copy()

y = employee_df["Attrition"]
X = employee_df.drop(columns=["Attrition"])

split = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

models = {
    "LogisticRegression": LogisticRegression(random_state=42),
    "SVC": SVC(probability=True, random_state=42),
    "RandomForest": RandomForestClassifier(random_state=42), 
    "DecisionTreeClassifier": DecisionTreeClassifier(random_state=42),
}

In [40]:
# Initialisation des dictionnaires pour stocker les scores
scores = []
predictions = {}
metrics = {
    'precision': {name: [] for name in models.keys()},
    'recall': {name: [] for name in models.keys()},
    'f1': {name: [] for name in models.keys()},
    'auc': {name: [] for name in models.keys()},
    'accuracy': {name: [] for name in models.keys()}
}

In [ ]:
# Évaluation des modèles
for train_index, test_index in split.split(X, y):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    
    for name, model in models.items():
        # Prédictions
        model.fit(X_train, y_train)
        predictions[name] = model.predict(X_test)    
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None
        
        # Calcul des métriques
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_proba) if y_proba is not None else np.nan
        accuracy = accuracy_score(y_test, y_pred)
        
        # Stockage des métriques
        metrics['precision'][name].append(precision)
        metrics['recall'][name].append(recall)
        metrics['f1'][name].append(f1)
        metrics['auc'][name].append(auc)
        metrics['accuracy'][name].append(accuracy)
        # Ajout des scores dans le DataFrame final
        scores.append({
            'Model': name,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1,
            'Accuracy': accuracy,
            'AUC': auc
        })

In [ ]:
scores_df = pd.DataFrame(scores)
scores_df_mean = scores_df.groupby('Model').mean()
scores_df_std = scores_df.groupby('Model').std()
scores_df_mean = scores_df_mean.sort_values(['F1 Score', 'Precision', 'Accuracy', 'Recall'], ascending=False) # TODO : change order
scores_df_std = scores_df_std.sort_values(['F1 Score', 'Precision', 'Accuracy', 'Recall'], ascending=True) # TODO : change order

print('Mean Scores')
print(scores_df_mean)
print('_'*50)
print('Standard Deviation')
print(scores_df_std)

### Optimiser le meilleur modèle

In [ ]:
# hyperparameter tuning
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC

svc_params_grid = {
    # Most important parameters
    'C': [0.1, 1, 10],
    'kernel': ['rbf', 'linear'],
    'gamma': ['scale', 'auto', 0.01],
    
    # Basic settings
    'class_weight': [None, 'balanced'],
    'random_state': [42],
    'probability': [True]
}

# creating model using GridSearchCV
SVC_model = GridSearchCV(
    estimator=SVC(probability=True),
    param_grid=svc_params_grid,
    cv=5,
    n_jobs=-1,
    scoring=['accuracy', 'f1', 'precision', 'recall', 'roc_auc'],
    refit='accuracy',
    verbose=0,
    error_score='raise',
    return_train_score=True
)

# Définition d'une grille simplifiée de paramètres
rf_params_grid = {
    # Paramètres essentiels
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'max_features': ['sqrt', 'log2'],
    
    # Paramètres de base
    'random_state': [42],
    'n_jobs': [-1],
    'class_weight': [None, 'balanced']
}

# Configuration du GridSearchCV avec des options avancées
RandomForest_model = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=rf_params_grid,
    cv=5,  # Validation croisée à 5 plis
    n_jobs=-1,  # Utilise tous les cœurs disponibles
    scoring=['accuracy', 'f1', 'precision', 'recall', 'roc_auc'],
    refit='accuracy',  # Réentraîne sur la meilleure métrique accuracy
    verbose=2,
    return_train_score=True,
    error_score='raise'
)

# model fitting
RandomForest_model.fit(X_train, y_train)
SVC_model.fit(X_train, y_train)

# getting best parameters
best_rf_model = RandomForest_model.best_estimator_
best_svc_model = SVC_model.best_estimator_

# displaying results
print("\nMeilleurs paramètres trouvés :")
print(f"\nRandomForest {RandomForest_model.best_params_}")
print(f"\nSVC {SVC_model.best_params_}")
print("\nMeilleur score de validation croisée:")
print(f"\nRandomForest {RandomForest_model.best_score_}")
print(f"\nSVC {SVC_model.best_score_}")


# displaying scores for metrics
rf_results = pd.DataFrame(RandomForest_model.cv_results_)
svc_results = pd.DataFrame(SVC_model.cv_results_)
print("\nRésultats détaillés pour la meilleure configuration:")
metrics = ['accuracy', 'f1', 'precision', 'recall', 'roc_auc']
for metric in metrics:
    mean_score = rf_results[f'mean_test_{metric}'].iloc[rf_results['rank_test_accuracy'].argmin()]
    std_score = rf_results[f'std_test_{metric}'].iloc[rf_results['rank_test_accuracy'].argmin()]
    print(f"RandomForest - {metric}: {mean_score:.3f} (+/- {std_score*2:.3f})")
    
    mean_score = svc_results[f'mean_test_{metric}'].iloc[svc_results['rank_test_accuracy'].argmin()]
    std_score = svc_results[f'std_test_{metric}'].iloc[svc_results['rank_test_accuracy'].argmin()]
    print(f"SVC - {metric}: {mean_score:.3f} (+/- {std_score*2:.3f})")